In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [4]:
df = pd.read_csv('tuulivoimatieto.csv')
df.columns = ['a','b','c','d','e']
df.head()

,a,b,c,d,e
0,2018-12-31 21:00:00,2018-12-31 22:00:00,2019-01-01 00:00:00,2019-01-01 01:00:00,1598.0
1,2018-12-31 22:00:00,2018-12-31 23:00:00,2019-01-01 01:00:00,2019-01-01 02:00:00,1552.0
2,2018-12-31 23:00:00,2019-01-01 00:00:00,2019-01-01 02:00:00,2019-01-01 03:00:00,1468.0
3,2019-01-01 00:00:00,2019-01-01 01:00:00,2019-01-01 03:00:00,2019-01-01 04:00:00,1427.0
4,2019-01-01 01:00:00,2019-01-01 02:00:00,2019-01-01 04:00:00,2019-01-01 05:00:00,1352.0
